## Convert old IMAS at NERSC to new IMAS 
June 21, 2024 

Venkitesh Ayyar

In [1]:
import numpy as np
import datetime
import os

import pymongo
import pprint
from bson.objectid import ObjectId

import yaml
import json
import xmltodict

In [2]:
# ## Convert json to yaml for easy viewing
# with open('gyrokinetics_new.json', 'r') as j:
#     GK_dict = json.loads(j.read())
    
# with open('gyrokinetics_new.yaml','w') as f:
#     yaml.dump(GK_dict,f,default_flow_style=False)

## Load both yaml files

In [3]:
fname='gyro_imas_old.yaml'
with open(fname,'r') as f:
    gk_dict = yaml.safe_load(f)
    
fname='gyro_imas_new.yaml'
with open(fname,'r') as f:
    GK_dict = yaml.safe_load(f)

In [4]:
print(gk_dict.keys(),'\n',GK_dict.keys())

dict_keys(['code', 'collisions', 'flux_surface', 'fluxes_integrated_norm', 'ids_properties', 'model', 'species', 'species_all', 'wavevector']) 
 dict_keys(['code', 'collisions', 'flux_surface', 'ids_properties', 'linear', 'max_repr_length', 'model', 'non_linear', 'normalizing_quantities', 'species', 'species_all', 'time', 'version'])


## Print dictionary hierarchy

In [5]:
def f_print_dict_struct(dict1,tab_count=0):
    '''
    Module to print dictionary hierarchy
    '''
    
    if isinstance(dict1,dict):
        for i,j in dict1.items():
            if not isinstance(j,dict):
                print('\t'*tab_count, i,'\t',type(j))
                if (isinstance(j,list) and len(j)>0):
                    if isinstance(j[0],dict):
                        print('\t'*tab_count,i,'[0]:')                    
                        f_print_dict_struct(j[0],tab_count+1)
            else: 
                # print('\t'*tab_count,"------------")
                print('\t'*tab_count,i,':')
                f_print_dict_struct(j, tab_count+1)

                
# f_print_dict_struct(gk_dict,0)

# f_print_dict_struct(GK_dict,0)
    

In [6]:
def f_get_dict_all_keys(dict1):
    '''
    Module to get lowest keys in dict hierarchy
    '''
    keys_lst =[] 
    
    if isinstance(dict1,dict):
        for i,j in dict1.items():
            if not isinstance(j,dict):
                keys_lst.append(i)
                if (isinstance(j,list) and len(j)>0):
                    if isinstance(j[0],dict):
                        keys_lst.extend( f_get_dict_all_keys(j[0]) )
            else: 
                keys_lst.extend( f_get_dict_all_keys(j) ) 

    return keys_lst

# keys_par2 = f_get_dict_all_keys(par2)

In [26]:
keys='code'
for key in gk_dict[keys]:
    print(key)
print('\n\n')
for key in GK_dict[keys]:
    print(key)

name
parameters
version



commit
description
library
max_repr_length
name
output_flag
parameters
repository
version


In [27]:
gk_dict.keys(),GK_dict.keys()

(dict_keys(['code', 'collisions', 'flux_surface', 'fluxes_integrated_norm', 'ids_properties', 'model', 'species', 'species_all', 'wavevector']),
 dict_keys(['code', 'collisions', 'flux_surface', 'ids_properties', 'linear', 'max_repr_length', 'model', 'non_linear', 'normalizing_quantities', 'species', 'species_all', 'time', 'version']))

### Mode

In [25]:
GK_dict[keys].keys(),gk_dict[keys].keys()

(dict_keys(['commit', 'description', 'library', 'max_repr_length', 'name', 'output_flag', 'parameters', 'repository', 'version']),
 dict_keys(['name', 'parameters', 'version']))

In [44]:
[key2.split('collision_')[-1] for key2 in gk_dict['model'].keys() if key2.startswith('collision')]

['ei_only',
 'energy_conservation',
 'finite_larmor_radius',
 'momentum_conservation',
 'pitch_only']

In [12]:
for key2 in GK_dict['model'].keys():
    if key2.startswith('include'):
        print(key2,GK_dict['model'][key2])

include_a_field_parallel 1
include_b_field_parallel 0
include_centrifugal_effects None
include_coriolis_drift None
include_full_curvature_drift None


In [14]:
for key2 in gk_dict['model'].keys():
    if key2.startswith('include'):
        print(key2,gk_dict['model'][key2])


include_a_field_parallel True
include_b_field_parallel False
include_centrifugal_effects False
include_full_curvature_drift False


In [53]:
collision_suffix = ['energy_conservation', 'finite_larmor_radius', 'momentum_conservation', 'pitch_only']
for suffix in collision_suffix:
    print(suffix,gk_dict['model']['collision_'+suffix])
    print(suffix,GK_dict['model']['collisions_'+suffix])


energy_conservation True
energy_conservation None
finite_larmor_radius False
finite_larmor_radius None
momentum_conservation True
momentum_conservation None
pitch_only False
pitch_only None


In [16]:
# # ## modification for species 
# key1='species'

# common_keys   = [i for i in gk_dict[key1][0].keys() if i in GK_dict[key1][0].keys()]
# old_only_keys = [i for i in gk_dict[key1][0].keys() if i not in GK_dict[key1][0].keys()]
# new_only_keys = [i for i in GK_dict[key1][0].keys() if i not in gk_dict[key1][0].keys()]

# print(common_keys,'\n\n',old_only_keys,'\n\n',new_only_keys)


In [21]:
for key2 in ['include_a_field_parallel', 'include_b_field_parallel', 'include_centrifugal_effects', 'include_full_curvature_drift'] :
    print(key2, gk_dict['model'][key2],GK_dict['model'][key2])

include_a_field_parallel True 1
include_b_field_parallel False 0
include_centrifugal_effects False None
include_full_curvature_drift False None


In [11]:
key1='model'

common_keys   = [i for i in gk_dict[key1].keys() if i in GK_dict[key1].keys()]
old_only_keys = [i for i in gk_dict[key1].keys() if i not in GK_dict[key1].keys()]
new_only_keys = [i for i in GK_dict[key1].keys() if i not in gk_dict[key1].keys()]

print(common_keys,'\n\n',old_only_keys,'\n\n',new_only_keys)

# for i in common_keys: 
#     print(i,'\t',gk_dict[key1][i],'\t',GK_dict[key1][i],'\n')

# for i in old_only_keys:
#     print(i, gk_dict[key1][i])
    
# for i in new_only_keys:
#     print(i, GK_dict[key1][i])

['include_a_field_parallel', 'include_b_field_parallel', 'include_centrifugal_effects', 'include_full_curvature_drift'] 

 ['collision_ei_only', 'collision_energy_conservation', 'collision_finite_larmor_radius', 'collision_momentum_conservation', 'collision_pitch_only', 'initial_value_run', 'non_linear_run', 'time_interval_norm'] 

 ['adiabatic_electrons', 'collisions_energy_conservation', 'collisions_finite_larmor_radius', 'collisions_momentum_conservation', 'collisions_pitch_only', 'include_coriolis_drift', 'max_repr_length', 'version']


In [17]:
# for key2 in  ['adiabatic_electrons', 'include_coriolis_drift', 'max_repr_length', 'version']:
#     print(key2, GK_dict['model'][key2])

for key2 in  ['collision_ei_only','initial_value_run', 'non_linear_run', 'time_interval_norm']: 
    print(key2, gk_dict['model'][key2])

collision_ei_only False
initial_value_run True
non_linear_run False
time_interval_norm []


### Modules

In [24]:
def f_collisions(old_gk_dict,new_gk_dict):
    ''' conversion for key: collisions '''

    main_key='collisions'
    old_gk,new_gk=old_gk_dict[main_key],new_gk_dict[main_key]
    
    dict1={}
    for key in ['max_repr_length','version']: 
        dict1[key] = new_gk[key]
    
    dict1['collisionality_norm']=old_gk['collisionality_norm']
    
    return dict1

def f_flux_surface(old_gk_dict,new_gk_dict):
    ''' conversion for key: flux_surface '''

    main_key='flux_surface'
    old_gk,new_gk=old_gk_dict[main_key],new_gk_dict[main_key]
    
    dict1={}
    ## Old keys not present 
    keys = ['triangularity_lower', 'triangularity_upper']
    
    ## New keys not present in old 
    keys = ['delongation_dr_minor_norm', 'dgeometric_axis_r_dr_minor', 'dgeometric_axis_z_dr_minor', 'max_repr_length', 'version']
    for key in keys:
        dict1[key] = new_gk[key]
    
    ## Common keys 
    common_keys = ['b_field_tor_sign', 'dc_dr_minor_norm', 'ds_dr_minor_norm', 'elongation', 'ip_sign', 'magnetic_shear_r_minor', 'pressure_gradient_norm', 'q', 'r_minor_norm', 'shape_coefficients_c', 'shape_coefficients_s']
    
    for key in common_keys:
        dict1[key] = old_gk[key]
        
    return dict1

def f_species_all(old_gk_dict,new_gk_dict):
    ''' conversion for key: species_all '''
    
    main_key='species_all'
    old_gk,new_gk=old_gk_dict[main_key],new_gk_dict[main_key]
    
    dict1={}
    ## Old keys not present 
    keys =  ['debye_length_reference', 'zeff'] 
    
    ## New keys not present in old 
    keys = ['angle_pol', 'debye_length_norm', 'max_repr_length', 'version']
    for key in keys:
        dict1[key] = new_gk[key]
    
    ## Common keys 
    common_keys = ['beta_reference', 'shearing_rate_norm', 'velocity_tor_norm'] 

    for key in common_keys:
        dict1[key] = old_gk[key]
        
    return dict1

def f_species(old_gk_dict, new_gk_dict):
    ''' conversion for key: species '''

    main_key='species'
    old_gk,new_gk=old_gk_dict[main_key][0],new_gk_dict[main_key][0]

    dict1={}
    ## Old keys not present 
    keys =  ['name'] 

    ## New keys not present in old 
    keys = ['max_repr_length', 'potential_energy_gradient_norm', 'potential_energy_norm', 'version']
    for key in keys:
        dict1[key] = new_gk[key]

    ## Common keys 
    common_keys = ['charge_norm', 'density_log_gradient_norm', 'density_norm', 'mass_norm', 'temperature_log_gradient_norm', 'temperature_norm', 'velocity_tor_gradient_norm'] 

    for key in common_keys:
        dict1[key] = old_gk[key]
    
    return [dict1]

def f_ids_properties(old_gk_dict,new_gk_dict):
    ''' conversion for key: ids_properties '''

    main_key='ids_properties'
    old_gk,new_gk=old_gk_dict[main_key],new_gk_dict[main_key]
    
    dict1={}
    ## Old keys not present 
    keys = ['creator', 'date'] 
    
    ## New keys not present in old 
    keys = ['creation_date', 'homogeneous_time', 'max_repr_length', 'name', 'occurrence_type', 'provenance', 'provider', 'version']
    for key in keys:
        dict1[key] = new_gk[key]
    
    ## Common keys 
    common_keys = ['comment'] 

    for key in common_keys[:]:
        dict1[key] = old_gk[key]
        
    ## Special fixes
    dict1['creation_data']=old_gk['date']
    dict1['provider'] = 'manual'
    
    ## Note: 'creator' not used. Should it be stored in 'name' ? 
    # dict1['name'] = old_gk['creator']

    return dict1

def f_model(old_gk_dict,new_gk_dict):
    ''' conversion for key: model '''

    ## To Do : Confirm what NoneType means for the True case in the new schema
    
    main_key='model'
    old_gk,new_gk=old_gk_dict[main_key],new_gk_dict[main_key]
    
    dict1={}
    ## Old keys not present 
    keys =  ['collision_ei_only','initial_value_run', 'non_linear_run', 'time_interval_norm']
    
    ## New keys not present in old 
    keys = ['adiabatic_electrons', 'include_coriolis_drift']
    for key in keys: 
        dict1[key] = None
    
    keys = ['max_repr_length', 'version']
    for key in keys:
        dict1[key] = new_gk[key]
    
    ## Common keys 
    common_keys = ['include_a_field_parallel', 'include_b_field_parallel']
    for key in common_keys: # Integer dtype
        dict1[key] = 1 if old_gk[key] else 0

    common_keys = ['include_centrifugal_effects', 'include_full_curvature_drift'] 
    for key in common_keys: ## None type
        dict1[key] = 1 if old_gk[key] else None
        
    ## common keys with slight modification in key name
    collision_suffix = ['energy_conservation', 'finite_larmor_radius', 'momentum_conservation', 'pitch_only']
    for suffix in collision_suffix: # old : collision_ (Bool) , new: collisions_ (None type)
        dict1['collisions_'+suffix] = 1 if gk_dict[main_key]['collision_'+suffix] else None 
        
    return dict1

# f_collisions(gk_dict,GK_dict)
# f_flux_surface(gk_dict,GK_dict)
# f_species_all(gk_dict,GK_dict)
# f_species(gk_dict,GK_dict)
# f_ids_properties(gk_dict,GK_dict)
f_model(gk_dict,GK_dict)



{'adiabatic_electrons': None,
 'include_coriolis_drift': None,
 'max_repr_length': 64,
 'version': {'idspy_internal_version': '2.0',
  'idspy_version': '034000.2.0',
  'imas_dd_git_commit': '845f1b30816f86a3cd4d53714dc56cdd307fdca1',
  'imas_dd_version': '03.40.00'},
 'include_a_field_parallel': 1,
 'include_b_field_parallel': 0,
 'include_centrifugal_effects': None,
 'include_full_curvature_drift': None,
 'collisions_energy_conservation': 1,
 'collisions_finite_larmor_radius': None,
 'collisions_momentum_conservation': 1,
 'collisions_pitch_only': None}

### Wavevector

In [ ]:
gk_dict['wavevector'][0].keys(), GK_dict['linear']['wavevector'][0].keys()


In [ ]:
gk_dict['wavevector'][0]['eigenmode'][0].keys()

In [ ]:
GK_dict['linear']['wavevector'][0]['eigenmode'][0].keys()

### Code

In [ ]:
gk_dict['code'].keys()

In [ ]:
gk_dict['code']['parameters']

In [ ]:
GK_dict['code'].keys()

In [ ]:
GK_dict['code']['library']

In [ ]:
par1 = gk_dict['code']['parameters']
par1.keys()

In [ ]:
key2 = 'charge'
key2 in par1.keys()

In [ ]:
# GK_dict['ids_properties']
par2 = xmltodict.parse(GK_dict['code']['parameters'])

In [ ]:
par2['root'].keys()

In [ ]:
[ky2 for ky1 in par2['root'].keys() for ky2 in par2['root'][ky1]]

In [ ]:
len(par1.keys()),len(keys_par2)

In [ ]:
common = [i for i in par1.keys() if i in keys_par2]
not_in_new = [i for i in par1.keys() if i not in keys_par2]
not_in_old = [i for i in keys_par2 if i not in par1.keys()]

In [ ]:
len(common),len(not_in_new),len(not_in_old)

In [ ]:
# f_print_dict_struct(par2,0)